In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 24 07:30:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## load drive data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/boty/project'

## install packages, imports

In [ ]:
!pip install import-ipynb
import import_ipynb

!pip install transformers
!pip install sacremoses
!pip install textdistance

!pip install deepl
!pip install fasttext
!pip install easynmt

In [ ]:
# load herbert

def load_herbert():
    tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
    model = AutoModel.from_pretrained("allegro/herbert-large-cased")
    return tokenizer, model

def load_polbert():
    model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
    tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
    return tokenizer, model




## proper code

In [ ]:
from transformers import AutoTokenizer, AutoModel, BertForMaskedLM, BertTokenizer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
import torch
# from easynmt import EasyNMT

import textdistance
import string
import numpy as np

import pandas as pd
from bs4 import BeautifulSoup
import requests
from ast import literal_eval
import json

from sklearn.metrics.pairwise import cosine_similarity

import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim

In [ ]:
from tools import DeepL, CONFIG, LanguageIdentification, M2M_translator
from scraping import Courses, StudyRules


importing Jupyter notebook from scraping.ipynb


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

class LanguageModel:

    def __init__(self, model_command, tokenizer_command):
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = model_command.to(self.device)
        self.tokenizer = tokenizer_command

    def encode_input(self, prompt):
        # addition of eos token
        input_ids = self.tokenizer.encode(prompt + self.tokenizer.eos_token, return_tensors="pt").to(self.device)
        # input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(device)
        return input_ids

    def generate_output(self, prompt):
        """
        generate answer from the model with sampling
        """
        input_ids = self.encode_input(prompt)
        # sampling to mimic more human behaviour
        output = self.model.generate(
            input_ids,
            # max_length=200,
            max_new_tokens=70,
            do_sample=True,
            top_k=15,
            temperature=0.75,
            pad_token_id=self.tokenizer.eos_token_id,

        )
        return self.tokenizer.batch_decode(output, skip_special_tokens=True)


In [ ]:
class Word2Vec_class:
    def __init__(self):
        self.model = api.load('word2vec-google-news-300')

# word2vec = Word2Vec_class()

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
URL = 'https://zapisy.ii.uni.wroc.pl/courses/semester/347'
courses = Courses(URL)
courses.get_courses_person('Jan Otop')

# model_name = "microsoft/DialoGPT-small"
# LM = LanguageModel(tokenizer_command=AutoTokenizer.from_pretrained(model_name), 
#                    model_command=AutoModelForCausalLM.from_pretrained(model_name))
LM2 = LanguageModel(model_command=BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill"),
                    tokenizer_command=BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill"))

LANGUAGE = LanguageIdentification()

config = CONFIG()

# chooose translator
translator = DeepL(config.config['DEEPL_KEY'])
m2m = M2M_translator()

In [ ]:
LM2.generate_output('How are you today?')

[" I'm doing well, how are you? Do you have any plans for the weekend? "]

In [ ]:
LM2.generate_output('How are you?')

[" I'm doing well. How about yourself? What are you up to today? I'm watching TV."]

In [ ]:
class CourseRecommender:

    def __init__(self):
        self.courses_english = self.read_to_json()


        self.emb_dict = self.create_embeddings(word2vec.model)

    def translate_descriptions(self, translator):
        """
        high usage of deepl API
        """
        courses_english = {}
        for course in courses.courses:
            current_lang = LANGUAGE.predict_en_pl(course['description'])
            if current_lang != 'en':
                en_descr = translator.translate(course['description'])
                courses_english[course['name']] = en_descr
            else:
                courses_english[course['name']] = course['description']
        return courses_english

    def process_stars(self, courses_english):
        for key, value in courses_english.items():
            if value[:2] == '**':
                i = 2
                while value[i:i+2] != '**':
                    i+=1
                print(value[:i+2])
                courses_english[key] = value[i+2:]
        return courses_english

    def save_to_json(self, courses_english):
        with open('courses_english.json', 'w') as fh:
            json.dump(courses_english, fh)

    def read_to_json(self):
        with open('courses_english.json', 'r') as fh:
            courses_english = json.load(fh)
        return courses_english

    def create_embeddings(self, word2vec_model):
        emb_dict = {}
        for key, value in self.courses_english.items():

            tokens = value.split()
            emb = np.zeros((len(tokens), 300))
            for i, token in enumerate(tokens):
                token = token.translate(str.maketrans('', '', string.punctuation)).lower()
                if token in word2vec_model:
                    emb[i] = word2vec_model[token]

            emb = emb.mean(axis=0)
            emb_dict[key] = emb
            # print(emb.shape)

        import pickle
        with open('emb_courses.pkl', 'wb') as fh:
            pickle.dump(emb_dict, fh)

        return emb_dict

    def find_best_course(self, course_in):
        cossim = {}
        for course_name in self.emb_dict.keys():
            if course_name != course_in:
                res = cosine_similarity(recommender.emb_dict[course_name].reshape(1, -1), recommender.emb_dict[course_in].reshape(1, -1))
                cossim[course_name] = res

        res = sorted(cossim.items(), key=lambda x: x[1], reverse=True)

        return [i[0] for i in res[:3]]

recommender = CourseRecommender()
recommender.find_best_course('Projekt: Deep Learning')

['Neural Networks and Natural Language Processing',
 'Advanced Data Mining',
 'Sieci komputerowe']

In [ ]:
# rule based system

QUESTION_DEGREE = 'Powiedz proszę, jaki stopień studiów Cię interesuje'


class RuleSystem:
    """
    rule based system working in Polish
    """

    def __init__(self):
        pass

    def check_if_contains(self, text: str, forms: list, error_size=2) -> bool:
        for f in forms:
            if f in text:
                return f
            elif textdistance.levenshtein.distance(' '.join(text.split()[:len(f.split())]), f) <= error_size:
                return f
        return ''

    def find_closest(self, text, candidates, surnames=False):

        # dist_dict = {}
        smallest_dist = 5
        closest = None
        for c in candidates:
            distance = textdistance.levenshtein.distance(text.lower(), c.lower())
            # print(distance, c)
            if distance < smallest_dist:
                smallest_dist = distance
                closest = c
        if surnames:
            if closest == None:
                # go through surnames only
                for c in candidates:
                    distance = textdistance.levenshtein.distance(text, c.split()[1])
                    if distance < smallest_dist:
                        smallest_dist = distance
                        closest = c
        return closest

    def basic_links(self, prompt):

        if prompt == 'zapisy':
            to_print = f"Zapisywać się na przedmioty i nie tylko można na stronie:\nhttps://zapisy.ii.uni.wroc.pl/'"
        elif prompt == 'ii':
            to_print = f"Strona główna Instytutu Informatyki:\nhttps://ii.uni.wroc.pl/'"
        
        return to_print

    def convert_type(self, type_course):
        mapping_abbreviations = {'sem': 'Seminarium', 'i2z': 'I2.Z - zastosowania inf.', 'i2t': 'I2.T - teoria inf.', 
                        'o1': 'Obowiązkowy 1', 'o2': 'Obowiązkowy 2', 'o3': 'Obowiązkowy 3', 
                        'human': 'Humanistyczno-społeczny', 'i1': 'Informatyczny 1', 'kinż': 'Kurs inżynierski',
                        'k1': 'K1 - kurs podstawowy', 'k2': 'K2 - kurs zaawansowany', 'iinź': 'Informatyczny inż.',
                        'inne': 'Inne'}
        type_course = type_course.lower()
        if type_course in mapping_abbreviations:
            return mapping_abbreviations[type_course]
        return type_course

    def choose_rule(self, prompt):
        prompt = prompt.translate(str.maketrans('', '', string.punctuation))
        prompt_lower = prompt.lower()

        basic_link = ['zapisy', 'jaka jest strona zapisów', 'gdzie mogę się zapisać', 'gdzie mogę zagłosować',
                      'na jakiejs stronie mogę się zapisać', 'na jakiejs stronie mogę zagłosować',
                      'gdzie mogę sprawdzić swój plan']

        check = self.check_if_contains(prompt_lower, basic_link)
        if check:
            to_print = self.basic_links('zapisy')
            return to_print

        # kursy prowadzacego
        courses_conduct_forms = ['jakie zajęcia prowadzi', 'co prowadzi', 'jakie kursy prowadzi', 'co wykłada',
                                 'jakie przedmioty prowadzi']
        check = self.check_if_contains(prompt_lower, courses_conduct_forms)
        if check:
            person = ' '.join(prompt.split()[len(check.split()):])
            closest = self.find_closest(person, courses.people, surnames=True)
            if closest:
                full_person = closest
            else:
                return f'Niestety, {person} nie prowadzi na Instytucie zajęć w tym semestrze.'

            found_courses = courses.get_courses_person(full_person)
            to_print = f'{full_person} prowadzi:\n'
            for c in found_courses:
                to_print += f"\t{c[0]}, {c[1]}, {c[2][0]} \n"
            return to_print

        # kursy z efektem, kursy z tagiem, kursy ze znacznikiem
        effects_prompts = ['jakie kursy mają efekt', 'jakie przedmioty mają efekt']
        tags_prompts = ['jakie kursy mają tag', 'jakie przedmioty mają tag']
        type_prompts = ['jakie kursy mają znacznik', 'jakie przedmioty mają znacznik']

        prompts = [effects_prompts, tags_prompts, type_prompts]
        filter_keys = ['allEffects', 'allTags', 'allTypes']

        fail_texts = ['Niestety, nie ma przedmiotów z efektem', 'Niestety, nie ma przedmiotów z tagiem',
                      'Niestety, nie ma przedmiotów ze znacznikiem']
        success_texts = ['Przedmioty z efektem', 'Przedmioty z tagiem',
                      'Przedmioty ze znacznikiem']

        get_methods = [courses.get_courses_effect, courses.get_courses_tag, courses.get_courses_type]

        for prompts_option, filter_key, fail_text, success_text, get_method in zip(prompts, filter_keys, 
                                                                        fail_texts, success_texts, get_methods):
            check = self.check_if_contains(prompt_lower, prompts_option)
            if check:
                current = ' '.join(prompt.split()[len(check.split()):])
                if filter_key == 'allTypes':
                    if len(current) <= 5:
                        current = self.convert_type(current)
                closest = self.find_closest(current, courses.filters[filter_key].values())
                if closest:
                    full_current = closest
                else:
                    return f'{fail_text} {current}.'

                found_courses = get_method(full_current)
                to_print = f"{success_text} {full_current}:\n"
                for c in found_courses:
                    to_print += f"\t{c},\n"
                return to_print

        # godziny przedmiotu
        courses_hours_forms = ['kiedy odbywa się', 'o której godzinie jest', 'o której godzinie są', 'gdzie jest',
                               'gdzie odbywa się', 'gdzie odbywają się']
        check = self.check_if_contains(prompt_lower, courses_hours_forms)
        if check:
            course = ' '.join(prompt.split()[len(check.split()):])
            closest = self.find_closest(course, courses.courses_names)
            if closest:
                full_course = closest
            else:
                return f'Niestety, przedmiot {course} nie jest prowadzony w tym semestrze.'

            found_course = courses.get_course_instances(full_course)
            to_print = f'Godziny i sale przedmiotu {full_course} to:\n'
            for c in found_course:
                to_print += f'\t{c[0]}\n'
                for inst in c[1]:
                    to_print += f"\t\t{inst[0]}, {inst[1]}\n"
            return to_print
        
        courses_prompts = ['gdzie mogę się zapisać na', 'jaka jest strona przedmiotu']
        check = self.check_if_contains(prompt_lower, courses_prompts)
        if check:
            course = ' '.join(prompt.split()[len(check.split()):])
            closest = self.find_closest(course, courses.courses_names)
            if closest:
                full_course = closest
            else:
                return f'Niestety, przedmiot {course} nie jest prowadzony w tym semestrze.'

            web_adr = "https://zapisy.ii.uni.wroc.pl/courses/" 
            web_adr += '-'.join(full_course.lower().translate(str.maketrans('', '', string.punctuation)).split())
            web_adr += '-202223-letni'
            return f"Strona przedmioty {full_course} to {web_adr}"

        # informacje o wymaganiach do skonczenia studiow
        courses_prompts = ['jakie są wymagania do zakończenia studiów']
        check = self.check_if_contains(prompt_lower, courses_prompts)
        if check:
            return QUESTION_DEGREE   


        courses_prompts = ['jaki przedmiot warto zrobić po', 'podobny przedmiot do']
        check = self.check_if_contains(prompt_lower, courses_prompts)
        if check:
            course = ' '.join(prompt.split()[len(check.split()):])
            closest = self.find_closest(course, courses.courses_names)
            if closest:
                full_course = closest
            else:
                return f'Niestety, przedmiot {course} nie jest prowadzony w tym semestrze.'

            to_print = f"Po zaliczeniu {full_course} warto się zastanowić na:\n"
            
            recommended = recommender.find_best_course(full_course)
            for c in recommended:
                to_print += f'\t{c}\n'
            return to_print


        return None

    def rules_details(self):
        study_type = ''
        req_type = ''

        prompt = input('Zosia: Jaki stopień studiów Cię interesuje?: ')
        prompt = prompt.translate(str.maketrans('', '', string.punctuation))
        prompt_lower = prompt.lower()

        # get stopien
        prompts_first = ['pierwszy', '1', 'licencjackie', 'inżynierskie']
        prompts_second = ['drugi', 'magisterskie', '2', 'ii']
        
        check1 = self.check_if_contains(prompt_lower, prompts_first, error_size=0)
        check2 = self.check_if_contains(prompt_lower, prompts_second, error_size=0)
        study_rules = StudyRules()
        if check1:
            chosen_rules = study_rules.data_first_degree()
        elif check2:
            chosen_rules = study_rules.data_second_degree()
        
        # first degree only
        if check1:
            degree_num = 1
            prompt1 = 'Interesują Cię studia licencjackie czy inżynierskie?: '
            prompts_option1 = ['licencjackie']
            prompts_option2 = ['inżynierskie']

        if check2:
            degree_num = 2
            prompt1 = 'Mówisz o studiach trzysemestralnych czy czterosemestralnych?: '
            prompts_option1 = ['trzysemestralnych']
            prompts_option2 = ['czterosemestralnych']

        prompt_degree = input(f'Zosia: {prompt1}')
        check11 = self.check_if_contains(prompt_degree, prompts_option1)
        check12 = self.check_if_contains(prompt_degree, prompts_option2)
        if check11:
            study_type = 0
            study_name = prompts_option1[0]
        elif check12:
            study_type = 1
            study_name = prompts_option2[0]

        prompt_req = input('Zosia: Chcesz dowiedzieć się o obowiązkach, ECTS czy innych wymaganiach?: ')
        if 'obowiązkach' in prompt_req.lower():
            to_print = chosen_rules.compulsory_courses
            print(f'Zosia: Przedmioty obowiązkowe dla stopień {degree_num}, {study_name}')
            for item in chosen_rules.compulsory_courses:
                print(f'\t{item}')
        elif 'ects' in prompt_req.lower():
            print(f'Zosia: Wymagane ECTS dla stopień {degree_num}, {study_name}')
            for key, value in chosen_rules.ects_numbers.items():
                print(f'\t{key}: {value[study_type]}')
        else:
            print(f'Zosia: Inne wymagania dla stopień {degree_num}, {study_name}')
            for key, value in chosen_rules.other_requirements.items():
                print(f'\t{key}: {value[study_type]}')
            

rules = RuleSystem()

# rules.choose_rule('gdzie mogę się zapisać')
# rules.choose_rule('o której godzinie jest bazy danych')
# rules.choose_rule('jaka jest strona Algorytmy i struktury danych')
# rules.choose_rule('jakie kursy mają efekt Sieci komputer')
# rules.choose_rule('jakie kursy mają tag data science')
# rules.choose_rule('jakie kursy mają znacznik i2.t')


In [ ]:
study_rules = StudyRules()
chosen_rules = study_rules.data_first_degree()
chosen_rules.compulsory_courses

['Analiza matematyczna',
 'Logika dla informatyków',
 'Algebra',
 'Metody programowania',
 'Analiza numeryczna',
 'Matematyka dyskretna (L)',
 'Algorytmy i struktury danych (L)']

In [ ]:

ENDING_KEY_WORD = 'Do zobaczenia!'
ENDING_FAREWELL = 'Do następnego!'

rules = RuleSystem()

def keep_constant_context_length(current_prompt, context, tokenizer, max_len=256):
    """
    keep constant and relatively short conversation context
    """
    current_context_len = len(tokenizer.encode(current_prompt)) # add eos ??
    new_context = []
    i = len(context) - 2
    # print(context)
    while current_context_len < max_len and i >= 0:
        current_context_len +=  len(tokenizer.encode("</s> <s>" + context[i]))
        new_context.append(context[i])
        i -= 1

    new_context.reverse()
    if new_context:
        return '</s> <s>'.join(new_context + \
                [current_prompt])
    else:        
        return '<s>' + current_prompt + '</s>'


def chatbot(lang_model, translator, translate=True):
    """
    main function to deliver chabot to the user
    """
    print(f'Witaj! Powiedz, o czym chciałbyś porozmawiać?')

    conversation_context = []
    chat_history_ids = []
    iter = 0
    while True:
        prompt = input('Ty :')

        # conversation ending
        if prompt == ENDING_KEY_WORD:
            print(ENDING_FAREWELL)
            break

        out = rules.choose_rule(prompt)
        if out:
            if out == QUESTION_DEGREE:
                rules.rules_details()
            else:
                 print(out)
            continue

        # translation part
        current_lang = LANGUAGE.predict_en_pl(prompt)
        if translate and current_lang != 'en':
            prompt = translator.translate(prompt, source_language='pl', target_language='en-us')

        conversation_context.append(prompt)

        prompt = keep_constant_context_length(prompt, conversation_context, lang_model.tokenizer, max_len=100)
                
        answer = lang_model.generate_output(prompt)
        answer = answer[0].strip()

        # second translation part
        if translate and current_lang != 'en':
            answer = translator.translate(answer, source_language='en', target_language='pl')
            print(' ', answer)

        # add answer to the context
        conversation_context.append(answer)
        iter +=1
        

In [ ]:
chatbot(LM2, translator, translate=True)

Witaj! Powiedz, o czym chciałbyś porozmawiać?
Ty :jaka jest strona przedmiotu bazy danych
Strona przedmioty Bazy danych to https://zapisy.ii.uni.wroc.pl/courses/bazy-danych-202223-letni
Ty :Do zobaczenia!
Do następnego!


In [ ]:
chatbot(LM2, translator, translate=True)

Witaj! Powiedz, o czym chciałbyś porozmawiać?
Ty :co tam słychać?
  Nic wielkiego, po prostu myślę o wszystkich dobrych chwilach, które miałem z moją rodziną, kiedy byłem młodszy.
Ty :o  gdzie mieszkałeś?
  Mieszkam w Stanach Zjednoczonych. A ty? Czym się zajmujesz w życiu?
Ty :Do zobaczenia!
Do następnego!


In [ ]:
chatbot(LM2, translator, translate=True)

Witaj! Powiedz, o czym chciałbyś porozmawiać?
Ty :jaki przedmiot warto zrobić po projekt: deep learning
Po zaliczeniu Projekt: Deep Learning warto się zastanowić na:
	Neural Networks and Natural Language Processing
	Advanced Data Mining
	Sieci komputerowe

Ty :jaki przedmiot warto zrobić po Architektury systemów komputerowych
Po zaliczeniu Architektury systemów komputerowych warto się zastanowić na:
	Metody programowania
	Advanced Distributed Algorithms
	Kurs administrowania systemem Linux

Ty :Course on Biomedical Image Analysis for Computer ScientistsCourse on Biomedical Image Analysis for Computer Scientists
Ty :jak się czujesz?
  Kocham komputery. Pracuję na nich na co dzień. Co robisz w pracy?
Ty :ja też kocham komputery!
  Ja też ją kocham! Uwielbiam fakt, że jest to gałąź matematyki
Ty :jaki przedmiot warto zrobić po Course on Biomedical Image Analysis for Computer Scientists
Po zaliczeniu Course on Biomedical Image Analysis for Computer Scientists warto się zastanowić na:
	Adva

In [ ]:
chatbot(LM2, translator, translate=True)

Witaj! Powiedz, o czym chciałbyś porozmawiać?
Ty :jakie są wymagania do zakończenia studiów
Powiedz proszę, jaki stopień studiów Cię interesuje
Jaki stopień studiów Cię interesuje?: 1
Interesują Cię studia licencjackie czy inżynierskie?: licencjackie
Chcesz dowiedzieć się o obowiązkach, ECTS czy innych wymaganiach?: ects
Wymagane ECTS dla stopień 1, licencjackie
	min. liczba ECTS: 180 ECTS
	za pracę dyplomową i egzamin: 10 ECTS
Ty :jakie przedmioty mają tag data science
Przedmioty z tagiem Data Science:
	Advanced Data Mining,
	Neural Networks and Natural Language Processing,
	Numerical Optimization,

Ty :a jakie przedmioty mają tag ekonomia?
Przedmioty z tagiem Ekonomia:
	Usługi finansowe,
	Usługi finansowe (w),

Ty :jakie zajęcia prowadzi Jurdziński?
Tomasz Jurdziński prowadzi:
	Advanced Distributed Algorithms, Wykłady, cz 10:00-12:00 (s. 105) 
	Advanced Distributed Algorithms, Ćwiczenia, cz 14:00-16:00 (s. 105) 

Ty :a jakie zajęcia prowadzi Leszek Grocholki?
Niestety, prowadzi Lesze